<a href="https://colab.research.google.com/github/ruchirlives/Python/blob/main/OpenAI_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load and run functions.py
%pip install --quiet openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
import requests
from google.colab import userdata

## OpenAI

In [ ]:
import openai
import os

from openai import OpenAI
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI')

def get_client():
    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )
    return client

def get_openai(prompt):
    client = get_client()

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o",
    )
    return chat_completion.choices[0].message.content

def get_embeddings(text):
    client = get_client()
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002",  # Adjust model name as necessary
    )
    embeddings = response.data[0].embedding  # Assuming a single input; adjust as needed
    return embeddings

def generate_summary(text):
    prompt = f"Please provide a summary of the following text in no more than 25 words:\n\n{text}"
    return get_openai(prompt)

def get_category(text, categories, prompt):
    client = get_client()

    prompt = f"{prompt} \
    Please classify the entry into one of the following categories, providing only the category by itself in the output. \
    \n\nCategories: {categories}\n\nText: {text}"

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who categorises stuff."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

# Extraction

In [ ]:
import ast

def generate_assertions(text):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{ASSERTION_PROMPT} Here is the text: " + text,
            }
        ],
        model="gpt-4o",
    )

    return response.choices[0].message.content

def extract_assertions(text):

    # generate assertions
    try:
        assertions = generate_assertions(text)
    except Exception as e:
        print('First try failed')
        print(e)
        try:
            assertions = generate_assertions(text)
        except Exception as e:
            print('Second try failed')
            print(e)
            return None

    # convert to list of dicts
    data_string = assertions.replace("```python", "").replace("```", "").strip()
    data_string = data_string.replace("'", r"\'")
    assertions = ast.literal_eval(data_string)
    return assertions

# Test first row
# assertions = process_assertions(blogs.iloc[82])

## _List functions in module

In [ ]:
# list all functions defined in this module only

import inspect
import sys

# Get the current module
current_module = sys.modules[__name__]

# List all functions and their arguments defined in this module
functions_with_args = {func: str(inspect.signature(getattr(current_module, func)))
                       for func in dir(current_module) if inspect.isfunction(getattr(current_module, func))}

# Display the function names with their arguments
for func, args in functions_with_args.items():
    print(f"{func}{args}")



['generate_summary', 'get_category', 'get_client', 'get_embeddings', 'get_openai']
